In [1]:
import torch
!pip show torch

def load_model_memory_mapped(model_path):
    # if torch.backends.mps.is_available():
    #     print(f"mps is available")
    #     device = torch.device('mps')
    # else:
    #     device = torch.device('cpu')
    return torch.load(
        model_path, map_location=torch.device('cpu'), mmap=True
    )


def compute_weight_sums(model):
    weight_sums = {}
    for name, param in model.named_parameters():
        # Compute the sum for rows and columns and store in a dictionary
        weight_sums[name] = {
            "row_sum": torch.sum(param, dim=0).cpu().numpy(),  # sum along rows
            "col_sum": torch.sum(param, dim=1).cpu().numpy(),  # sum along columns
        }
    return weight_sums


def main(model_path):
    model = load_model_memory_mapped(model_path)
    weight_sums = compute_weight_sums(model)

    # Output the sums for visualization or further analysis
    for name, sums in weight_sums.items():
        print(f"Parameter: {name}")
        print(f"Row Sum: {sums['row_sum']}")
        print(f"Column Sum: {sums['col_sum']}")
        print("========================================")

from os.path import expanduser
model_path = expanduser("~/Downloads/consolidated.00.pth")

Name: torch
Version: 1.13.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /Users/markuszhang/.pyenv/versions/3.9.17/lib/python3.9/site-packages
Requires: typing-extensions
Required-by: 


In [2]:
model = load_model_memory_mapped(model_path)
model.keys()

TypeError: 'mmap' is an invalid keyword argument for Unpickler()

In [ ]:
for key in model.keys():
  size = model[key].size()
  print(f"{key}: {size}")

: 

In [ ]:
def angle_between_two_vectors(A: torch.Tensor, B: torch.Tensor) -> float:
		"""Find the angle between two vectors in Eucledian space.
  
  	Args:
			A (torch.Tensor): First vector
   		B (torch.Tensor): Second vector
     
    Returns:
			float: Angle in radians of range [0, pi]
    """
		dot = torch.dot(A, B)
		length = torch.norm(A) * torch.norm(B)
		cos = dot / length
		angle = torch.acos(cos)
		return angle

: 

In [ ]:
from tqdm import trange

def angles_in_unit_vectors_of_matrix_sequential(matrix: torch.Tensor) -> torch.Tensor:
    """Find the angles between all the unit (column) vectors of a matrix.

    Args:
            matrix (torch.Tensor): Matrix of shape (N, M)

    Returns:
            torch.Tensor: Angles in radians of shape (N, N)
    """

    angles = []
    for i in trange(matrix.size(1)):
        for j in range(matrix.size(1)):
            angle = angle_between_two_vectors(matrix[:, i], matrix[:, j])
            angles.append(angle)
    return torch.tensor(angles).view(matrix.size(1), matrix.size(1))

: 

In [ ]:
def angles_in_unit_vectors_of_matrix(matrix: torch.Tensor) -> torch.Tensor:
    """Find the angles between all the unit (column) vectors of a matrix.

    Args:
        matrix (torch.Tensor): Matrix of shape (N, M)

    Returns:a
        torch.Tensor: Angles in radians of shape (M, M)
    """
    # Normalize columns to ensure they are unit vectors
    normalized_matrix = matrix / matrix.norm(dim=0)

    # Compute the dot product between all pairs of columns
    dot_products = torch.matmul(normalized_matrix.t(), normalized_matrix)

    # Ensure dot products are within [-1, 1] due to potential numerical issues
    dot_products = torch.clamp(dot_products, min=-1.0, max=1.0)

    # Compute the angles between columns using the arccosine function
    angles = torch.acos(dot_products)

    return angles

: 

In [ ]:
matrix_name = "layers.0.attention.wq.weight"

A = model[matrix_name][:,0]
B = model[matrix_name][:,1]

angle = angle_between_two_vectors(A, B)

angles_test = angles_in_unit_vectors_of_matrix_sequential(model[matrix_name])
angles_test[0,1], angle

: 

: 